In [1]:
%pip install tensorflow tensorflow-cpu opencv-python matplotlib scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold


#### Set GPU memory growth

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

#### Define data directories

In [4]:
data_dir_good = 'Good_UI'
data_dir_bad = 'Bad_UI'

# Set image parameters

In [5]:
image_size = (256, 256)
batch_size = 32

#### Define data augmentation

In [6]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
])

#### Create ImageDataGenerator for loading and augmenting images

In [7]:
datagen = ImageDataGenerator(preprocessing_function=data_augmentation)

#### Load and prepare the good image dataset

In [8]:
good_dataset = datagen.flow_from_directory(
    data_dir_good,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

Found 0 images belonging to 0 classes.


#### Load and prepare the bad image dataset

In [9]:
bad_dataset = datagen.flow_from_directory(
    data_dir_bad,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

Found 0 images belonging to 0 classes.


#### Combine both datasets

In [10]:
combined_dataset = tf.data.experimental.sample_from_datasets([good_dataset, bad_dataset])

Instructions for updating:
Use `tf.data.Dataset.sample_from_datasets(...)`.


AttributeError: 'DirectoryIterator' object has no attribute 'element_spec'

#### Perform cross-validation using K-fold

In [ ]:
k = 5  # Number of folds
num_samples = len(combined_dataset)
fold_indices = list(range(num_samples))
kfold = KFold(n_splits=k)

In [ ]:
for fold, (train_indices, val_indices) in enumerate(kfold.split(fold_indices)):
print(f"Fold: {fold + 1}")

### # Split data into training and validation sets

In [ ]:
train_data = combined_dataset.take(train_indices)
val_data = combined_dataset.skip(train_indices).take(val_indices)

#### Define and compile the model

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(256, 256, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

#### Train the model

In [ ]:
history = model.fit(train_data, epochs=20, validation_data=val_data)

#### Evaluate the model

In [ ]:
test_loss, test_accuracy = model.evaluate(val_data)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")